In [5]:
import numpy as np

Defining the Markov chain transition matrix; needs to be a n by n matrix (array) with all positive entries whose rows sum to one

In [6]:
transitionMatrix = np.array([[.3, .5, .2], [.5, .3, .2], [.5, .2, .3]]);

Here we defined a transition matrix which implicitly defines a Markov chain. The transition matrix is
$$\mathbb{P} = \begin{pmatrix} 
          .3 & .5 & .2 \\
          .5 & .3 & .2 \\
          .5 & .2 & .3
          \end{pmatrix}
          $$
The $i^{th}$ row and $j^{th}$ column in this transition matrix, denoted $[\mathbb{P}]_{i,j}$, represents the probability of transitioning from state $i$ to state $j$. For example $[\mathbb{P}]_{0,1} = 0.5$ tells us there is a 50% chance of transitioning from state 0 to state 1

In [8]:
transitionMatrix = np.transpose(transitionMatrix)

While the above notation is customary it does require state vectors, $\alpha_{n}$, to be applied on the left-hand side. Since $Av=b$ is more common than $vA=b$ we shall solve this problem by transposing the matrix.

In [9]:
numStates = len(transitionMatrix[0]);

Here we define a variable named numStates. As the number suggest it contains the number of states in the Markov chain

Our ultimate goal is to solve for $\mathbb{E}[N(i_1, ..., i_n) | X_{0} = r]$. In persuit of that goal we need to find the long-run proportions for each state. Long-run proportions is the unique vector, $\pi$, whose entries are all positive, sum to one and is the solution to $\mathbb{P} \pi = \pi$, or simply, $(\mathbb{P} - Id_{n}) \pi = 0$ where $Id_{n}$ is the n by n identity matrix.


In [10]:
longrunMatrix = np.array(transitionMatrix, copy = True)
identityMatrixNumStates = np.identity(numStates)

longrunMatrix = longrunMatrix - identityMatrixNumStates

Here we made a copy of the transition matrix (for reasons that are soon to follow), longrunMatrix, and assigned a variable, identityMatrixNumStates, to hold $Id_{n}$. Finally we assigned the variable longrunMatrix to $(\mathbb{P} - Id_{n})$ as per the equation above. Our system of linear equations can be represented as
$$\begin{align}
  \pi_{0} &= P_{0,0} \pi_{0} + ... + P_{n,0} \pi_{n} \\
  \pi_{1} &= P_{0,1} \pi_{0} + ... + P_{n,1} \pi_{n} \\
  &\vdots\\
  \pi_{n} &= P_{0,n} \pi_{0} + ... + P_{n,n} \pi_{n}
\end{align}$$
Adding the first n-1 equations to the last one yields
$$\begin{align}
  \pi_{0} &= P_{0,0} \pi_{0} + ... + P_{n,0} \pi_{n} \\
  \pi_{1} &= P_{0,1} \pi_{0} + ... + P_{n,1} \pi_{n} \\
  &\vdots\\
  \sum_{i=1}^{i=n} \pi_{i} &= \sum_{i=1}^{i=n} (P_{0,i}) \pi_{0} + ... + (P_{n,i}) \pi_{n}
\end{align}$$
But since $\sum_{i=1}^{i=n} P_{j,i} = 1$ for all j (from any state j you have to transition into one of the n states) this equation reduces further to
$$\begin{align}
  \pi_{0} &= P_{0,0} \pi_{0} + ... + P_{n,0} \pi_{n} \\
  \pi_{1} &= P_{0,1} \pi_{0} + ... + P_{n,1} \pi_{n} \\
  &\vdots\\
  \sum_{i=1}^{i=n} \pi_{i} &= \sum_{i=1}^{i=n} \pi_{i} \implies 0 = 0
\end{align}$$
This equation is useless. Thus we have n variables and n-1 equations. We cannot hope to have a unique solution in this underconstrained situation. Luckily we have the added equation
$$\sum_{i=1}^{i=n} \pi_{i} = 1$$
Thus we must alter one row of the matrix $(\mathbb{P} - Id_{n})$ and the corresponding entry of the solution vector to represent this equation. This can be done by replacing a row with all 1s and the corresponding entry with a 1.

In [11]:
equationSwap = np.full(shape = numStates, fill_value = 1)
longrunMatrix[numStates-1] = equationSwap

In [12]:
solutionVec = equationSwap - equationSwap
solutionVec[numStates-1] = 1

Here the variable equationSwap is our vector of all 1s. It replaces the last row of longrunMatrix and is the reason we made a copy of the variable transitionMatrix (we do not wish to mutate this variable). The variable solutionVec is, as the name suggest, the vector we will solve for. With our equations in place all we have to do is linearly solve for the vector $\pi$

In [13]:
longrunVec = np.linalg.solve(longrunMatrix, solutionVec)

With our $\pi$ vector found we can move on to the next piece of the puzzle. Our next goal is to find the value of $\mu(i,j)$, the mean time spent waiting to visit state $j$ given you start in state $i$. I shall refer to the matrix filled with the entries $\mu(i,j)$ as $\mu$ and the individual entries as $\mu_{i,j}$. These values can be solved for with the system of equations
$$\mu_{i,j} = 1 + \sum_{i=1; i \neq j}^{i=n} P_{i,k} \mu_{k,j}$$
or equivalently ,
$$\begin{align}
  1 &= (1 - P_{0,0})  \mu_{0,j} - P_{0,1} \mu_{1,j} - \dotsc - (0) \mu_{j,j} - \dotsc - P_{0,n} \mu_{n,j} \\
  1 &= -P_{1,0} \mu_{0,j} + (1 - P_{1,1}) \mu_{1,j} - \dotsc - (0) \mu_{j,j} - \dotsc - P_{1,n} \mu_{n,j} \\
  &\vdots\\
  1 &= -P_{n,0} \mu_{0,j} - P_{n,1} \mu_{1,j} - \dotsc - (0) \mu_{j,j} - \dotsc - (1 - P_{n,n}) \mu_{n,j}
  \end{align}$$
If we let $\mu_{j}$ be the $j^{th}$ column of the matrix $\mu$, $\mathbb{P}^{(j)}$ be the tranistion matrix whose $j^{th}$ column is all zeroes and $1_{n}$ be a n-dimensional vector of all ones. Then this equation can be represented in matrix form as
$$(Id_{n} - \mathbb{P}^{(j)}) \mu_{j} = 1_{n}$$

In [14]:
oneVector = np.full(shape = numStates, fill_value = 1.0)

Here we assigned the vector $1_{n}$ to the variable oneVector

In [20]:
zeroVector = np.full(shape = numStates, fill_value = 0.0)
meanTransitionsBetweenStates = np.copy(identityMatrixNumStates)

for i in range(0, numStates):
    #Making a copy of transition matrix so the transition matrix isn't mutated#
    systemOfEquationsToStatei = np.array(transitionMatrix, copy = True)
    #Replcing ith row with all zeroes as per the linear equations above#
    systemOfEquationsToStatei[i] = zeroVector
    #Transposing matrix so the zeroes are all in the ith column. I made sure this was the "right" column i.e.# 
    #I didn't end up changing a row/column when compared to the system of equations#
    systemOfEquationsToStatei = np.transpose(systemOfEquationsToStatei)
    systemOfEquationsToStatei = (identityMatrixNumStates - systemOfEquationsToStatei)
    #Using a numpy built in to linearly solve for \mu_{i}#
    meanTransitionsToStatei = np.linalg.solve(systemOfEquationsToStatei, oneVector)
    #Adding \mu_{i} to a matrix to house all the \mu_{i,j} values#
    meanTransitionsBetweenStates[i] = meanTransitionsToStatei

Here we assigned the n-dimensional zero vector to the variable zeroVector. We make a copy of the n by n indentity matrix to the variable meanTransitionsBetweenStates. This is merely to have an n by n matrix to hold the values of $\mu_{i,j}$. Since each $\mu_{j}$ vector requires a different matrix $(\mathbb{P}^{(j)})$ to solve for it we loop over j. In the loop we first make a copy of the variable transitionMatrix (our $\mathbb{P}$) and assign it to the matrix systemOfEquationsToStatei. This is so we can have a matrix we feel free to mutate. Our next step is mutating systemOfEquationsToStatei by replacing the $i^{th}$ row with all zeroes. One can check that for our system of equations to be correct we must transpose systemOfEquationsToStatei. Now systemOfEquationsToStatei is $\mathbb{P}^{(i)}$. Next we replace systemOfEquationsToStatei with $(Id_{n} - \mathbb{P}^{(i)})$. Finally we solve the equation $(Id_{n} - \mathbb{P}^{(i)}) \mu_{i} = 1_{n}$ and assigned it to the variable meanTransitionsToStatei. Finally the values of meanTransitionsToStatei is assigned to the $i^{th}$ column of meanTransitionsBetweenStates

In [16]:
meanTransitionsBetweenStates = np.transpose(meanTransitionsBetweenStates)

One can check that for the entries in meanTransitionBetweenStates to match up with our matrix $\mu$ we need to transpose meanTransitionBetweenStates. Thus, meanTransitionBetweenStates is replaced with its transpose. 

In [17]:
pattern = ["0", "1", "0", "1", "0"]

Here we set a pattern of interest to the variable pattern. In this case we are interested in mean time it would take for the Markov chain defined above to visit the states 0 then 1 then 0 then 1 then 0.

Let $s_{1}$ be the size of of the largest cycle in the pattern and $i_1, \dotsc, i_{s_{1}}$ be the first $s_{1}$ elements of the pattern. We wish to determine the value of $\mathbb{E}[A(i_1, ..., i_{s_{1}})]$, the mean time until the pattern occurs given that last observed states are $i_1, ..., i_{s_{1}}$. Ross's book tells us that 
$$E[A(i_1, ..., i_{s_{1}})] = \frac{1}{\pi_{i_1} P_{i_1, i_2}  \dotsm P_{i_{s_{1}} -1, i_{s_{1}}}}$$. 
The reason for this is described in Ross's book. 

Once $\mathbb{E}[A(i_1, ..., i_{s_{1}})]$ is determined we record it and find the value of $s_{2}$ the size of the largest cycle in the first $i_1, ..., i_{s_1}$ elements of the pattern. We again wish to find the value of $\mathbb{E}[A(i_1, ..., i_{s_{1}})]$, the mean time until the pattern of the first $i_1, ..., i_{s_1}$ elements occurs given that last observed states are $i_1, ..., i_{s_{2}}$. This process is repeated until the value of the largest cycle, $s_{n}$, is zero. Finally we record the value of $\sum_{j=1}^{j=n-1} \mathbb{E}[A(i_1, ..., i_{s_{j}}]$

In [18]:
def meanTimeForPatternWithCycles(pattern, longrunVec, transitionMatrix, runningSum = 0):
    patternLength = len(pattern)
    cycleLength = 0
    startingStateOfPattern = int(pattern[0])
    meanTimeToPatternGivenCycle = longrunVec[startingStateOfPattern]
    for i in range(0, patternLength - 1):
        currentState = int(pattern[i])
        futureState = int(pattern[i+1])
        meanTimeToPatternGivenCycle = (meanTimeToPatternGivenCycle)*(transitionMatrix[futureState][currentState])
    try:
        meanTimeToPatternGivenCycle = 1/(meanTimeToPatternGivenCycle)
    except:
        print("The pattern is impossible")
        return None
        
    runningSum = runningSum + meanTimeToPatternGivenCycle
#boolean to make sure we check if the current position in the list is#
#is equal to the last item in the list. The boolean becomes false if and only if#
#there exist a cycle (the largest cycle)#
    stillChecking = True

    for i in range(0, patternLength-1):
    #Creating mutable variable keeping track of current position in list. 
    #Starting from the second to last element and moving down will ensure that 
    #the first cycle found is the largest cycle#
        current = patternLength - 2 - i
    
    #Creating a variable. This variable will keep track how many letters in 
    #the pattern are the same starting from the "current" variable and compared
    #to the last element#
        currentPatternLength = 0

        while pattern[current] == pattern[-1 - (currentPatternLength)] and stillChecking:
        #Setting up the check to see if the letter before the currrent letter 
        #is the same as the current letters to the left of the end of the word#
            current = current - 1
            currentPatternLength = currentPatternLength + 1
        #Checking to see if we found the largest cycle#
            if current == -1:
            #Recording length of largest cycle#
                cycleLength = currentPatternLength
            #Breaking out of both loops#
                i = patternLength - 1
                stillChecking = False
    
    if cycleLength == 0:
        return [pattern, runningSum]
    else:
        cyclePattern = pattern[:cycleLength]
        return meanTimeForPatternWithCycles(cyclePattern, longrunVec, transitionMatrix, runningSum)


In [19]:
def meanTimeForPattern(pattern, longrunVec, transitionMatrix, meanTransitionsBetweenStates, startingState):
    [cyclelessPattern, runningSum] = meanTimeForPatternWithCycles(pattern, longrunVec, transitionMatrix)
    patternStart = int(cyclelessPattern[0]);
    patternEnd = int(cyclelessPattern[-1])
    startingState = int(startingState);
    return meanTransitionsBetweenStates[startingState, patternStart] + runningSum - meanTransitionsBetweenStates[patternEnd, patternStart]


print(meanTimeForPattern(pattern, longrunVec, transitionMatrix, meanTransitionsBetweenStates, startingState = "0"))

63.0
